<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/mmd_branch/notebooks/Federated_learning_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and data

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from torchvision.datasets import CIFAR100
from torch.utils.data import Subset
import numpy as np
import copy
import random
import torch
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title In this block we import the test set of CIFAR100 to evaluate the global model
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:03<00:00, 44.0MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


# Collecting data of models we want to aggregate

In [ ]:
#@title Client's log file has been loaded in this block so we can use it in next steps
clients_data = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")
clients_data.head()

,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
0,0,dino_vits16,28b80fff-61ff-445b-8c19-6210b0fefd2e,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.960880,0.162077,0.218887,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,66.262507,2025-05-27 09:34:10,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
1,1,dino_vits16,66ce83c7-9027-469e-b937-4040d8ef3d02,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",97.555012,0.157862,0.183864,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,65.349265,2025-05-27 09:35:23,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
2,8,dino_vits16,34fe6e15-49ea-4530-a9c2-91fb18b9c71a,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.471883,0.179580,0.226160,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,67.385976,2025-05-27 10:15:59,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
3,14,dino_vits16,e00bd69c-dade-4936-bbd5-beb9f9138bd0,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.349633,0.174917,0.211031,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,62.720521,2025-05-27 10:17:15,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
4,9,dino_vits16,a026faa9-5fa9-4da4-9455-c5fbcd0ade4e,4f20c1df-2f32-48b8-8289-3acac23e314f,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.410758,0.177857,0.207068,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,1,63.000698,2025-05-27 10:23:15,/content/drive/MyDrive/MLDL_FederatedLearning/...,/content/drive/MyDrive/MLDL_FederatedLearning/...


In [ ]:
initial_model_name = "572ab5a3-b275-4e53-aab8-61adca93077a" #@param{"type":"string"}

In [ ]:

filter =  clients_data['initial_model_name']== initial_model_name
filtered_clients_data = clients_data[filter] # Using filter to collect clients with specified initial model
params = filtered_clients_data[['backbone',
                                    'num_of_clients',
                                    'splitting_method',
                                    'size_of_dataset']]
params = dict(params.iloc[0])
print("Number of all trained clients:", len(clients_data))
print("Number of clients after filtering:", len(filtered_clients_data))
contributors = [] # contributors is being used to store the name of models which contributes in aggregation
for i  in filtered_clients_data['model_name'].values:
  contributors.append(i)
print("Contributors:", contributors)
filtered_clients_data.head()

Number of all trained clients: 40
Number of clients after filtering: 20
Contributors: ["dino_vits16_c0000_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0001_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0002_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0003_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0004_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0005_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0006_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0007_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0008_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_c0009_r0002_methodiid_Ncall_Nk20_sr{'train': 0.8, 'test': 0.2}_ds50000.pth", "dino_vits16_

,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,size_of_dataset,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs
20,0,dino_vits16,dino_vits16_c0000_r0002_methodiid_Ncall_Nk20_s...,572ab5a3-b275-4e53-aab8-61adca93077a,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",99.449878,0.093691,0.108017,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,2,67.674202,2025-06-02 17:58:03,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
21,1,dino_vits16,dino_vits16_c0001_r0002_methodiid_Ncall_Nk20_s...,572ab5a3-b275-4e53-aab8-61adca93077a,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.594132,0.116127,0.139266,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,2,69.524983,2025-06-02 17:59:21,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
22,2,dino_vits16,dino_vits16_c0002_r0002_methodiid_Ncall_Nk20_s...,572ab5a3-b275-4e53-aab8-61adca93077a,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.777506,0.110634,0.129001,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,2,67.867842,2025-06-02 18:00:36,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
23,3,dino_vits16,dino_vits16_c0003_r0002_methodiid_Ncall_Nk20_s...,572ab5a3-b275-4e53-aab8-61adca93077a,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",99.083130,0.099777,0.115313,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,2,67.308623,2025-06-02 18:01:50,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN
24,4,dino_vits16,dino_vits16_c0004_r0002_methodiid_Ncall_Nk20_s...,572ab5a3-b275-4e53-aab8-61adca93077a,/content/drive/MyDrive/MLDL_FederatedLearning/...,20,"accuracy,loss,train_loss",98.716381,0.105556,0.127096,...,50000,1636,410,"{'train': 0.8, 'test': 0.2}",all,2,67.062268,2025-06-02 18:03:04,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN


In [ ]:
#@title Dino Model
import torch
import torch.nn as nn

class DinoClassifire(nn.Module):
  def __init__(self, backbone, num_classes:int=100, device=None):
    super(DinoClassifire, self).__init__()
    self.backbone = torch.hub.load('facebookresearch/dino:main', backbone)

    #We need to freaze thhe parameters of bakbone first so we can train only on the head layer(output layer)
    for param in self.backbone.parameters():
      param.requires_grad = False

    #determine the Device
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.backbone.to(device)

    #To detect the output feature dimontion of backbone we run  Dummy forward pass
    with torch.no_grad():

      dummy_input = torch.randn(1,3,224,224).to(device)
      dummy_out = self.backbone(dummy_input)


      if isinstance(dummy_out, tuple):
        dummy_out = dummy_out[0]
      elif isinstance(dummy_out, dict):
        dummy_out = dummy_out.get("x_norm_clstoken", next(iter(dummy_out.values())))

      #If the output is 3D (B, T, D), we assume first token is the [CLS] token.
      if dummy_out.dim() == 3:
        dummy_feature = dummy_out[:,0]
      else:
        dummy_feature = dummy_out
      feature_dim = dummy_feature.shape[1]
      print("Detected feature dimontion:", feature_dim)


      #Difineing the classification Head
      self.head = nn.Linear(feature_dim, num_classes)

      #Ensure the head is trainable.
      for param in self.head.parameters():
        param.requires_grad = True

  def forward(self,x):

    #pass the input through the backbone
    features = self.backbone(x)

    if isinstance(features, tuple):
      features = features[0]
    elif isinstance(features, dict):
      features = features.get("x_norm_clstoken", next(iter(features.values())))


    # If featers are retuened as (B, T, D), use the first token
    if features.dim() == 3:
      cls_token = features[:,0]
    else:
      cls_token = features
    logits = self.head(cls_token)

    return logits

In [ ]:
# @title `get_model` function retrieves and loads the models of filtered clients
def get_model(paths,sample_sizes, backbone):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = DinoClassifire(backbone=backbone, num_classes=100, device=device) # Loading an initial custom dino model
  for index in range(len(paths)):
    state_dict = torch.load(paths.iloc[index]) # load state dict regarding the client number 'index'
    model.load_state_dict(state_dict) # set the state dict based on client
    model.to(device)
    yield (model,sample_sizes.iloc[index]) # this command throws model one at the time so less time and resouces will be used

# Aggregation functions

Implemented algorithm:

*   FebAvg
*   FebAvgM
*   EMA



In [ ]:
#@title <h2>FebAvg</h2>
def feb_avg(df):
  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  global_head = None # This variable stores the weights we want to modify

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if global_head is None:
        global_head = copy.deepcopy(model.head.state_dict())
        global_model = copy.deepcopy(model)
        for k in global_head.keys():
          global_head[k].zero_() # This command sets the tensor to zero

      for k in global_head.keys():
        global_head[k] += model.head.state_dict()[k] * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights

    global_model.head.load_state_dict(global_head) # A model with modified head will be assignd

  return global_model



In [ ]:
def feb_avg_m(df, momentum_coefficient=0.9, cliping_range=5,  momentum_vector_path= None):

  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  delta = None # `delta` is variable that keep the average of clients

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta is None:

        global_model = copy.deepcopy(model)
        global_head = copy.deepcopy(model.head.state_dict())
        delta = { k: torch.zeros_like(v) for k, v in global_head.items() } # A dict with structure of the model that we want to modify will be generated


      client_head = model.head.state_dict()
      for k in delta.keys():

        delta[k] += (client_head[k] - global_head[k]) * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights


    # In this section we calculate the momentum_vector

    if momentum_vector_path is None :
      momentum_vector = copy.deepcopy(delta) # In first round the `delta` as momentum vector
    else:
      momentum_vector = torch.load(momentum_vector_path) #In rounds > 1 momentum vector is requerd to be loaded from drive
      for k in delta.keys():
        momentum_vector[k] = momentum_coefficient * momentum_vector[k]  + delta[k]  # Using the the formula of FebAvgM we calculate the momentum vector







    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector.keys():
      global_head[k] = torch.clamp(global_head[k] + momentum_vector[k], -cliping_range, cliping_range) #After adding momentum vector the last global model we use clamp function we insure to keep momentum vector in boundary

    global_model.head.load_state_dict(global_head)

  return global_model, momentum_vector # We return momentum_vector to save it and use for next aggregation steps



In [ ]:
def ema(df, decay=0.9, cliping_range=5, momentum_vector_path= None):

  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  delta = None # `delta` is variable that keep the average of clients

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta is None:

        global_model = copy.deepcopy(model)
        global_head = copy.deepcopy(model.head.state_dict())
        delta = { k: torch.zeros_like(v) for k, v in global_head.items() } # A dict with structure of the model that we want to modify will be generated


      client_head = model.head.state_dict()
      for k in delta.keys():

        delta[k] += (client_head[k] - global_head[k]) * (sample_size / total_samples)  # Each weight will be assgin by average of all clients weights


    # In this section we calculate the momentum_vector
    if momentum_vector_path is None :
      momentum_vector = copy.deepcopy(delta) # In first round the `delta` as momentum vector it is also possible to use aa zero tensor
    else:
      momentum_vector = torch.load(momentum_vector_path)
      for k in delta.keys():
        momentum_vector[k] = decay * momentum_vector[k]  + (1- decay) * delta[k]  # Using the the formula of EMA we calculate the momentum vector


    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector.keys():
      global_head[k] = torch.clamp(global_head[k] + momentum_vector[k], -cliping_range, cliping_range)#After adding momentum vector the last global model we use clamp function we insure to keep momentum vector in boundary

    global_model.head.load_state_dict(global_head)

  return global_model, momentum_vector # We return momentum_vector to save it and use for next aggregation steps



In [ ]:
# @title This function will evaluate the model.</br> The outputs are loss and accuracy
def evaluation(model, data_loader):
  criterion = nn.CrossEntropyLoss()
  model.eval()
  correct = 0
  total = 0
  test_loss = 0

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  with torch.no_grad():
    for images, labels in  data_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      _, prediction = torch.max(outputs.data,1)
      loss = criterion(outputs, labels)
      test_loss += loss.item() * labels.size(0)

      total += labels.size(0)
      correct += (prediction == labels).sum().item()
    accuracy = 100 * correct / total
    loss = test_loss / total
    return accuracy, loss



In [ ]:
# @title <h2>`next_id`</h2> This function generates a unique name for model. `uuid4` does not generate duplicated but we are using a fixed `seed` hence we insure this name does not already exists.

from uuid import uuid4
import os
def next_id(log_path):
  if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    while True:
      uuid = str(uuid4())
      if uuid not in df["model_name"].values:
        return uuid
  else:
    return str(uuid4())

In [ ]:
from datetime import datetime

def get_current_time():
  now = datetime.now()

  formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S") # Format the date and time as a string

  return formatted_date_time

def global_model_name_path_generator():

  model_name = next_id("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")

  path = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/" + model_name + ".pth"

  return model_name, path



In [ ]:
def del_model(model_name):
  log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  filter = log_df["model_name"] == model_name
  if not filter.any():
    print(f"recored ({model_name}) not found.")
    return
  if os.path.exists(log_df[filter]["path"].values[0]):
    os.remove(log_df[filter]["path"].values[0])
  else:
    print("model not found")

  if log_df[filter]["momentum_vector_path"].values[0] is not None and os.path.exists(log_df[filter]["momentum_vector_path"].values[0]):
    os.remove(log_df[filter]["momentum_vector_path"].values[0])
  else:
    print("momentum vector not found")
  log_df = log_df[~filter]
  log_df.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)
# del_model("cd156602-8620-4d9a-b1ee-d831103e9f53")

In [ ]:
#@title use this block to modify the global log file
temp = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
temp["aggregation_method"] = np.nan
temp = temp[['backbone',
              'num_of_clients',
              'splitting_method',
              'aggregation_method',
              'Measurement_criteria',
              'accuracy',
              'loss',
              'size_of_dataset',
              'train_test_ratio',
              'classes',
              'round_number',
              'num_of_participants',
              'model_name',
              'prev_global_model_name',
              "contributors",
              'path',
              "momentum_vector_path",
              'path_to_subsets',
              'path_to_class_combs',
              'time'
               ]]
# temp.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)
temp.head()
del temp

In [ ]:
#@title <h1>Select aggregation method </h1>
#@markdown This value will be auto assigned in case the initial model of filtered clients have been aggregated with a spicific aggregation function in the previous rounds

aggregation_method = "EMA"   #@param["FebAvg","FebAvgM", "EMA" ]

prev_agg_method = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
filter = prev_agg_method["model_name"] == initial_model_name
prev_agg_method = prev_agg_method[filter]
prev_agg_method = prev_agg_method["aggregation_method"].values[0]
aggregation_method = aggregation_method if prev_agg_method is None else prev_agg_method


print(f"{aggregation_method} has been selected as the Aggregation function.")

FebAvg has been selected as the Aggregation function.


In [ ]:

if aggregation_method == "FebAvg":
  global_model = feb_avg(filtered_clients_data)
elif aggregation_method == "FebAvgM":
  global_model,momentum_vector = feb_avg_m(filtered_clients_data)
elif aggregation_method == "EMA":
  global_model,momentum_vector = ema(filtered_clients_data)



model_name, path = global_model_name_path_generator()

test_accracy, test_loss= evaluation(global_model, test_loader)
print(f"Accurace:{test_accracy:.2f}")
print(f"Loss:{test_loss:.2f}")

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


Detected feature dimontion: 384


<ipython-input-8-503d3d34f3e8>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(paths.iloc[index])


Accurace:74.74
Loss:0.93


In [ ]:
import os
log_path = "/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv"

prev_global_model_name = filtered_clients_data["initial_model_name"].values[0]
global_model_log = filtered_clients_data.drop(["client_id","train_loss","client_train_size","client_test_size","duration",],axis=1)
global_model_log = global_model_log.iloc[0]
global_model_log["num_of_participants"] = len(filtered_clients_data)
global_model_log["prev_global_model_name"] = initial_model_name
global_model_log["model_name"]= model_name
global_model_log["accuracy"] = test_accracy
global_model_log["loss"] = test_loss
global_model_log["time"] = get_current_time()
global_model_log["path"] = path
global_model_log["Measurement_criteria"] = "accuracy,loss"
global_model_log["contributors"] = contributors
global_model_log["aggregation_method"] = aggregation_method

if aggregation_method == "FebAvg":
  global_model_log["momentum_vector_path"] = None

elif aggregation_method == "FebAvgM" or aggregation_method == "EMA":
  global_model_log["momentum_vector_path"] = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/momentun_vectors/MV_"+ model_name + ".pt"
  torch.save(momentum_vector, global_model_log["momentum_vector_path"])


global_model_log = pd.DataFrame(global_model_log).T
global_model_log = global_model_log[['backbone',
              'num_of_clients',
              'splitting_method',
              'aggregation_method',
              'Measurement_criteria',
              'accuracy',
              'loss',
              'size_of_dataset',
              'train_test_ratio',
              'classes',
              'round_number',
              'num_of_participants',
              'model_name',
              'prev_global_model_name',
              "contributors",
              'path',
              "momentum_vector_path",
              'path_to_subsets',
              'path_to_class_combs',
              'time'
               ]]

flag = False
if os.path.exists(path):
  print("Model already exist")
  flag = True

if os.path.exists(log_path):
  global_log_df = pd.read_csv(log_path)
  if model_name in global_log_df["model_name"].values:
    print("record already exist")
    flag = True
  if not flag:
    global_model_log.to_csv(log_path, mode='a', header=False, index=False)
else:
  if not flag:
    global_model_log.to_csv(log_path, index=False, header=True)


if not flag:
  torch.save(global_model.state_dict(),  path )
  if aggregation_method == "FebAvgM" or aggregation_method == "EMA":
    torch.save(momentum_vector, global_model_log["momentum_vector_path"].values[0])


In [ ]:
global_model_log.head()

,backbone,num_of_clients,splitting_method,aggregation_method,Measurement_criteria,accuracy,loss,size_of_dataset,train_test_ratio,classes,round_number,num_of_participants,model_name,prev_global_model_name,contributors,path,momentum_vector_path,path_to_subsets,path_to_class_combs,time
20,dino_vits16,20,i.i.d. sharing,FebAvg,"accuracy,loss",74.74,0.929976,50000,"{'train': 0.8, 'test': 0.2}",all,2,20,3eaf89e4-c0a5-4ba8-8bb3-deaaad5c8129,572ab5a3-b275-4e53-aab8-61adca93077a,[dino_vits16_c0000_r0002_methodiid_Ncall_Nk20_...,/content/drive/MyDrive/MLDL_FederatedLearning/...,None,/content/drive/MyDrive/MLDL_FederatedLearning/...,NaN,2025-06-02 18:52:21
